# Corporacion Favorita Grocery Sales Forecasting
# Prediction Behavior Visual Comparison
### Display random time series along with the predictions for different models to determine their relevance and find out some hints for ensembling

In [1]:
# imports
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# load train, test and items
types = {'id': 'int64','item_nbr': 'int32','store_nbr': 'int8','unit_sales': 'float32','onpromotion': bool,}
df_train= pd.read_csv("clean_train.csv", parse_dates = ['date'], dtype = types, infer_datetime_format = True)
types = {'id': 'int64','item_nbr': 'int32','store_nbr': 'int8','onpromotion': bool,}
df_test = pd.read_csv("data/test.csv", parse_dates = ['date'], dtype = types, infer_datetime_format = True)
df_items= pd.read_csv("data/items.csv")

a
b


In [4]:
"""
    LIST THE PREDICTION FILES YOU WANT TO DISPLAY HERE
"""

config = {'LGBM': ('lgb_OIL.csv', 'red'), \
          'ETR': ('EXTRA_OIL.csv', 'brown'), \
          'FOREST': ('FOREST_OIL.csv', 'green'), \
          'LOGMA': ('LOGMA_526.csv', 'navy'), \ 
          'LSTM': ('lstm_SQR.csv', 'magenta'),\
          'RIDGE': ('RIDGE_OIL.csv', 'cyan'),
          'CAT': ('CAT_OIL.csv', 'orange'),
           # best ensemble: weighted average of LightGBM, Extra Trees, Random Forest, 
           #                                    LSTM and moving averages.
          'BEST>': ('LGEXRFLSMA_OILL.csv', 'black')
         }

subs = {}
colors = {}
for key, conf in config.items():
    subs[key] = df_test.merge(pd.read_csv(conf[0]), on='id')
    colors[key] = conf[1]

SyntaxError: unexpected character after line continuation character (<ipython-input-4-0ed554d83bc8>, line 6)

In [ ]:
picked = [] # keep track of previous comparisons when random

# Compare predictions of different models on a random timeline

In [ ]:
_LAST_X_DAYS = 80 # define time frame to display for train data, if None : display all available data
_RANDOM = True # change to True to display random comparison
if _RANDOM:
    df_s = df_test.sample(1)
    _store, _item = df_s['store_nbr'].values[0], df_s['item_nbr'].values[0]
    picked.append((_store, _item))
    print("STORE :: %d" % _store)
    print("ITEM :: %d" % _item)
else:
    _store, _item = 34, 103665
    
df_train_c = df_train[(df_train.store_nbr == _store) & (df_train.item_nbr == _item)].filter(items=['date', 'unit_sales']).set_index('date')

u_dates = [d for d in df_train.date.unique() if pd.Timestamp(d) >= df_train_c.index.min()]
df_train_c = df_train_c.reindex(u_dates)
df_train_c.unit_sales = df_train_c.unit_sales.fillna(0.0)

f, ax = plt.subplots(2, 1, figsize=(20, 20))

if df_train_c.shape[0] > 0:
    if _LAST_X_DAYS == None:
        df_train_c.rename(columns={"unit_sales": "unit_sales_train"}).plot(ax=ax[0], color='black')
    else:
        df_train_c[-_LAST_X_DAYS:].rename(columns={"unit_sales": "unit_sales_train"}).plot(ax=ax[0], color='black')

for k in subs.keys():
    df_plot = subs[k][(subs[k].store_nbr == _store) & (subs[k].item_nbr == _item)].filter(items=['date', 'unit_sales']).set_index('date').rename(columns={"unit_sales": k})
    df_plot.plot(ax=ax[0], color=colors[k])
    df_plot.plot(ax=ax[1], color=colors[k])

plt.show()
_store, _item